* degeneracy point identification
* external flux solo (ohne ng)
* choice of basis : 
* ridge transformation

In [1]:
from DiSuQ.Torch import models
import scqubits
from torch import tensor
from torch.linalg import inv
from numpy import arange,linspace,meshgrid,array,log,pi,around
from DiSuQ.utils import plotCompare,plotHeatmap
from torch import set_num_threads
from DiSuQ.Torch.components import indE,capE,indEnergy
set_num_threads(32)

### analytic feature recovering : optimal parameters

In [2]:
L_ = indE(1e-7); L0 = indE(5*1e-9); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(5*1e-16); C_ = capE(1e-14*5) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(5*1e-16/4); CJ_ = capE(1e-14/2) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 25. ; J_ = 5. ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 1.6346151294983178 34.32691771946467
Capacitance Bound(GHz): 0.38740458615415185 39.12786320156934
Shunt Bound(GHz): 3.8740458615415188 158.83588032320228
Junction Bound(GHz): 5.0 30.0


In [3]:
El,Ec,Ej,EcJ = 1.1*L_,1.1*C_,J0+.9*J_,CJ0+.9*CJ_

In [4]:
EcJ/Ec , Ej/Ec , Ej/El , EcJ/El

(371.8181818181818, 69.22525642871614, 16.406419672875916, 88.12109116845791)

In [5]:
.25/2/indEnergy(El) # upper bound on potential energy

8.873152571901118

### Flux profile Spectrum

Charge Basis assure flux profile symmetry with respect to $\Delta \Phi_x \Delta \Phi_y$ \
Subsequent analysis is in Flux basis, for better convergence, to demonstrate circuit features.

In [6]:
pairs = {'Lx':'Ly','Cx':'Cy','Jx':'Jy','CJx':'CJy'}

#### charge regime circuit

In [7]:
basis = [9,9,9]
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ,sparse=False,symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))

In [8]:
Nx,Ny  = 50,1
PhiX = linspace(.45,.55,Nx,endpoint=True)
PhiY = linspace(0,1,Ny,endpoint=True)
flux_range = meshgrid(PhiX,PhiY)
flux_range = array((flux_range[0].flatten(),flux_range[1].flatten())).T
flux_manifold = [flux for flux in tensor(flux_range)]

In [9]:
H_LC = circuit.chargeHamiltonianLC()
H_J = circuit.josephsonCharge

In [ ]:
qE0,(qEx1,qEx2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=False)

In [ ]:
H_LC = circuit.fluxHamiltonianLC()
H_J = circuit.josephsonFlux

In [ ]:
fE0,(fEx1,fEx2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=False)

In [ ]:
plotCompare(flux_range[:,0],{'charge E0':qE0,'flux E0':fE0,'charge Ex1':qEx1,'flux Ex1':fEx1,
                            'charge Ex2':qEx2,'flux Ex2':fEx2},'Spectrum','flux profile','energy(GHz)')

In [ ]:
#utils.plotHeatmap(Ex1.reshape(Ny,Nx),PhiX,PhiY,'Zero-Pi:E01','phiX','phiY',size=(800,800),export='pdf')

In [ ]:
#utils.plotHeatmap((Ex1-Ex2).reshape(Ny,Nx),PhiX,PhiY,'Zero-Pi:E12','phiX','phiY',size=(800,800),export='pdf')

#### flux regime circuit

In [ ]:
basis = [8,8,8]
flux_point = ['Lx','Ly']
circuit = models.zeroPi(basis,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ,sparse=False,symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))

In [ ]:
Nx,Ny  = 25,1
PhiX = linspace(0,1,Nx,endpoint=True)
PhiY = linspace(0,1,Ny,endpoint=True)
flux_range = meshgrid(PhiX,PhiY)
flux_range = array((flux_range[0].flatten(),flux_range[1].flatten())).T
flux_manifold = [flux for flux in tensor(flux_range)]

##### charge basis

In [ ]:
H_LC = circuit.chargeHamiltonianLC()
H_J = circuit.josephsonCharge

In [ ]:
qE0,(qEx1,qEx2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=False)

##### kerman basis

In [ ]:
basis = {'O':[15,15],'I':[],'J':[8]}
circuit.basis = basis

In [ ]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [ ]:
kE0,(kEx1,kEx2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=False)

##### ridge basis

In [ ]:
circuit = models.zeroPi(basis,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ,sparse=False,symmetry=True,_L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0))

In [ ]:
basis = {'O':[10],'I':[],'J':[8,8]}
circuit.basis = basis

In [ ]:
circuit.R = tensor([[1,-1,-1],[-1,1,-1],[-1,-1,1]])/2.

In [ ]:
inv(circuit.R)@circuit.R

In [ ]:
circuit.No,circuit.Ni,circuit.Nj = 1,0,2
circuit.Lo_,circuit.C_ = circuit.transformComponents()

In [ ]:
Cn_,Ln_ = circuit.componentMatrix()

In [ ]:
Cn_

In [ ]:
inv(circuit.R)@Cn_@circuit.R

In [ ]:
H_LC = circuit.kermanHamiltonianLC()
H_J = circuit.kermanHamiltonianJosephson

In [ ]:
rE0,(rEx1,rEx2) = circuit.spectrumManifold(flux_point,flux_manifold,H_LC,H_J,excitation=[1,2],grad=False)

In [ ]:
charge_degeneracy = log(qEx2/qEx1)
flux_degeneracy = log(fEx2/fEx1)
kerman_degeneracy = log(kEx2/kEx1)
ridge_degeneracy = log(rEx2/rEx1)

In [ ]:
plotCompare(flux_range[:,0],{'charge D':charge_degeneracy,'flux D':flux_degeneracy,
                             'kerman D':kerman_degeneracy,'ridge D':ridge_degeneracy,
                             'charge Ex1':qEx1,'flux Ex1':fEx1,'kerman Ex1':kEx1,'ridge Ex1':rEx1,
                            'charge Ex2':qEx2,'flux Ex2':fEx2,'kerman Ex2':kEx2,'ridge Ex2':rEx2},
            'Spectrum','flux profile','energy(GHz)')

### Node Variables